In [ ]:
import pandas as pd
import snorkel
import numpy as np
from scipy.special import rel_entr


In [ ]:
df1 = pd.read_csv("../data2/euro_sat/forest.csv")
df2 = pd.read_csv("../data2/euro_sat/grasslands.csv")
df3 = pd.read_csv("../data2/euro_sat/industry.csv")
df4 = pd.read_csv("../data2/euro_sat/lakes.csv")
df5 = pd.read_csv("../data2/euro_sat/residential.csv")
df6 = pd.read_csv("../data2/euro_sat/river.csv")

df_final=df1.merge(df2, on="origin", how = "inner")
df_final = df_final.merge(df3, on="origin", how="inner")
df_final = df_final.merge(df4, how="inner", on="origin")
df_final = df_final.merge(df5, how="inner", on="origin")
df_final = df_final.merge(df6, how="inner", on="origin")

df_final = df_final.rename({'sum(isIntersect_forest)':'intersect_forest',
                 'sum(isTouches_forest)':'touch_forest',
                 'sum(isContain_forest)':'contain_forest',
                 'sum(isOverlaps_forest)':'overlap_forest',
                 'min(distance_forest)':'distance_forest',
                 
                 'sum(isIntersect_grasslands)':'intersect_grassland',
                 'sum(isTouch_grasslands)':'touch_grassland',
                 'sum(isContain_grasslands)':'contain_grassland',
                 'sum(isOverlaps_grassland)':'overlap_grassland',
                 'min(distance_grasslands)':'distance_grassland',
                 
                 'sum(isIntersect_industry)':'intersect_industry',
                 'sum(isTouch_industry)':'touch_industry',
                 'sum(isContain_industry)':'contain_industry',
                 'sum(isOverlaps_industry)':'overlap_industry',
                 'min(distance_industry)':'distance_industry',
                 
                 'sum(isIntersect_lakes)':'intersect_lake',
                 'sum(isTouch_lakes)':'touch_lake',
                 'sum(isContain_lakes)':'contain_lake',
                 'sum(isOverlaps_lake)':'overlap_lake',
                 'min(distance_lakes)':'distance_lake',
                 
                 'sum(isIntersect_residential)':'intersect_residential',
                 'sum(isTouch_residential)':'touch_residential',
                 'sum(isContain_residential)':'contain_residential',
                 'sum(isOverlaps_residential)':'overlap_residential',
                 'min(distance_residential)':'distance_residential',
                 
                 'sum(isIntersect_river)':'intersect_river',
                 'sum(isTouch_river)':'touch_river',
                 'sum(isContain_river)':'contain_river',
                 'sum(isOverlaps_river)':'overlap_river',
                 'min(distance_river)':'distance_river',
                 
                }, axis=1)




In [ ]:
ground_truth = pd.read_csv("../data2/euro_sat/textural_features/textural.csv")
# ground_truth = ground_truth[['origin', 'class']]
def map_origin(x):
    return x.split("/")[-1]

ground_truth['origin'] = ground_truth['origin'].apply(map_origin)

df_final = df_final.merge(ground_truth, how="inner", on="origin")
df_final.count()

In [ ]:
columns = ['distance_forest', 'distance_grassland', 'distance_industry', 'distance_lake', 'distance_residential', 'distance_river']

for col in columns:
    df_final[col] = round((df_final[col] - df_final[col].min()) / (df_final[col].max() - df_final[col].min()),3)


df_final.head()

In [ ]:
col_feat = ["ndwi", "mdwi", "ndmi", "ndvi", "awei", "bi", "rvi", "glcm_contrast", "glcm_dissimilarity", "glcm_homogeneity", "glcm_energy", "glcm_correlation", "glcm_ASM"]

for col in col_feat:
    df_final[col] = (df_final[col] - df_final[col].min()) / (df_final[col].max() - df_final[col].min())


df_final.head()

In [ ]:
''' Combined manual featues '''
from snorkel.labeling import labeling_function

from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
import random

y_train = df_final

y_label = df_final['class']


LAND = 1
WATER = 0

@labeling_function()
def lf1(x):
    if round(x.ndwi,3) > 0.249:
        return WATER
    else:
        return LAND

@labeling_function()
def lf2(x):
    if round(x.mdwi, 3) > 0.433:
        return WATER
    else:
        return LAND
    

@labeling_function()
def lf3(x):
    if round(x.ndmi, 4) > 0.4299:
        return WATER
    else:
        return LAND
    

@labeling_function()
def lf4(x):
    if round(x.ndvi, 3) > 0.711:
        return LAND
    else:
        return WATER
    

@labeling_function()
def lf5(x):
    if round(x.awei, 2) > 0.22:
        return WATER
    else:
        return LAND
    

@labeling_function()
def lf6(x):
    if x.bi > 0.57:
        return LAND
    else:
        return WATER
    

@labeling_function()
def lf7(x):
    if round(x.rvi, 2) > 0.34:
        return LAND
    else:
        return WATER
    

@labeling_function()
def lf8(x):
    if x.glcm_contrast > 0.37:
        return LAND
    else:
        return WATER
    

@labeling_function()
def lf9(x):
    if round(x.glcm_energy,2) > 0.52:
        return LAND
    else:
        return WATER
    
    
@labeling_function()
def lf10(x):
    if round(x.glcm_homogeneity,2) > 0.64:
        return WATER
    else:
        return LAND
    
    
@labeling_function()
def lf11(x):
    if round(x.glcm_correlation, 2) > 0.57:
        return WATER
    else:
        return LAND
    
@labeling_function()
def lf12(x):
    if round(x.glcm_ASM ,3)> 0.469:
        return WATER
    else:
        return LAND

@labeling_function()
def lf13(x):
    if round(x.glcm_dissimilarity,3) > 0.349:
        return LAND
    else:
        return WATER
    
@labeling_function()
def lf14(x):
    if x.distance_forest < 0.225:
        return WATER
    else:
        return LAND
    
@labeling_function()
def lf15(x):
    if x.distance_river < 0.812:
        return WATER
    else:
        return LAND
    
@labeling_function()
def lf16(x):
    if x.distance_lake < 0.465:
        return WATER
    else:
        return LAND
    
@labeling_function()
def lf17(x):
    if x.distance_residential<0.001:
        return LAND
    else:
        return WATER

@labeling_function()
def lf18(x):
    if x.distance_industry < 0.001:
        return LAND
    else:
        return WATER
@labeling_function()
def lf19(x):
    if x.distance_grassland < 0.019:
        return LAND
    else:
        return WATER
    
lfs = [
    lf1,
    lf2,
    lf3,
    lf4,
    lf5,
    lf6,
    lf7,
    lf8,
    lf9,
    lf10,
    lf11,
    lf12,
    lf13,
    lf14,
    lf15,
    lf16,
    lf17,
    lf18,
    lf19     ]

applier = PandasLFApplier(lfs)
L_train = applier.apply(y_train)
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, seed=123, lr=0.01, log_freq=10, n_epochs=300)

predicted_labels = label_model.predict(L_train)
print(label_model.score(L_train, y_label, metrics=["f1_micro"])['f1_micro'])

In [ ]:
''' Semi-automatic'''

@labeling_function()
def lf1(x):
    if round(x.ndwi,3) > 0.520:
        return WATER
    else:
        return LAND

@labeling_function()
def lf2(x):
    if round(x.mdwi, 3) > 0.580:
        return WATER
    else:
        return LAND
    

@labeling_function()
def lf3(x):
    if round(x.ndmi, 4) > 0.570:
        return WATER
    else:
        return LAND
    

@labeling_function()
def lf4(x):
    if round(x.ndvi, 3) > 0.480:
        return LAND
    else:
        return WATER
    

@labeling_function()
def lf5(x):
    if round(x.awei, 2) > 0.400:
        return WATER
    else:
        return LAND
    

@labeling_function()
def lf6(x):
    if x.bi > 0.43:
        return LAND
    else:
        return WATER
    

@labeling_function()
def lf7(x):
    if round(x.rvi, 2) > 0.1:
        return LAND
    else:
        return WATER
    

@labeling_function()
def lf8(x):
    if x.glcm_contrast > 0.50:
        return LAND
    else:
        return WATER
    

@labeling_function()
def lf9(x):
    if round(x.glcm_energy,2) > 0.60:
        return LAND
    else:
        return WATER
    
    
@labeling_function()
def lf10(x):
    if round(x.glcm_homogeneity,2) > 0.77:
        return WATER
    else:
        return LAND
    
    
@labeling_function()
def lf11(x):
    if round(x.glcm_correlation, 2) > 0.82:
        return WATER
    else:
        return LAND
    
@labeling_function()
def lf12(x):
    if round(x.glcm_ASM ,2)> 0.17:
        return WATER
    else:
        return LAND

@labeling_function()
def lf13(x):
    if round(x.glcm_dissimilarity,3) > 0.210:
        return LAND
    else:
        return WATER

@labeling_function()
def lf14(x):
    if x.distance_forest < 0.1:
        return WATER
    else:
        return LAND
    
@labeling_function()
def lf15(x):
    if x.distance_river < 0.19999999999999996:
        return WATER
    else:
        return LAND
    
@labeling_function()
def lf16(x):
    if x.distance_lake < 0.18999999999999995:
        return WATER
    else:
        return LAND
    
@labeling_function()
def lf17(x):
    if x.distance_residential<0.1:
        return LAND
    else:
        return WATER

@labeling_function()
def lf18(x):
    if x.distance_industry < 0.1:
        return LAND
    else:
        return WATER
@labeling_function()
def lf19(x):
    if x.distance_grassland < 0.1:
        return LAND
    else:
        return WATER
    
lfs = [
    lf1,
    lf2,
    lf3,
    lf4,
    lf5,
    lf6,
    lf7,
    lf8,
    lf9,
    lf10,
    lf11,
    lf12,
    lf13,
    lf14,
    lf15,
    lf16,
    lf17,
    lf18,
    lf19     ]

applier = PandasLFApplier(lfs)
L_train = applier.apply(y_train)
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, seed=123, lr=0.01, log_freq=10, n_epochs=300)

predicted_labels = label_model.predict(L_train)
print(label_model.score(L_train, y_label, metrics=["f1_micro"])['f1_micro'])

In [ ]:

@labeling_function()
def lf1(x):
    if x.ndwi <= 0.4724326878786087:
        if x.glcm_homogeneity <= 0.790214866399765:
            if x.glcm_ASM <= 0.5219392478466034:
                return LAND
            else:  # if glcm_ASM > 0.5219392478466034
                return WATER
        else:  # if glcm_homogeneity > 0.790214866399765
            if x.glcm_energy <= 0.5710632801055908:
                return LAND
            else:  # if glcm_energy > 0.5710632801055908
                return WATER
    else:  # if ndwi > 0.4724326878786087
        if x.glcm_homogeneity <= 0.7721803486347198:
            if x.glcm_ASM <= 0.4479398876428604:
                return LAND
            else:  # if glcm_ASM > 0.4479398876428604
                return WATER
        else:  # if glcm_homogeneity > 0.7721803486347198
            if x.glcm_energy <= 0.5140046775341034:
                return LAND
            else:  # if glcm_energy > 0.5140046775341034
                return WATER
            
@labeling_function()
def lf2(x):
    if x.glcm_homogeneity <= 0.7576185464859009:
        if x.glcm_homogeneity <= 0.7345372438430786:
            if x.ndvi <= 0.47253021597862244:
                return WATER
            else:  # if ndvi > 0.47253021597862244
                return LAND
        else:  # if glcm_homogeneity > 0.7345372438430786
            if x.glcm_ASM <= 0.4424889385700226:
                return WATER
            else:  # if glcm_ASM > 0.4424889385700226
                return LAND
    else:  # if glcm_homogeneity > 0.7576185464859009
        if x.glcm_homogeneity <= 0.8126062154769897:
            if x.glcm_ASM <= 0.44565123319625854:
                return LAND
            else:  # if glcm_ASM > 0.44565123319625854
                return WATER
        else:  # if glcm_homogeneity > 0.8126062154769897
            if x.glcm_dissimilarity <= 0.11553601920604706:
                return WATER
            else:  # if glcm_dissimilarity > 0.11553601920604706
                return LAND
            
@labeling_function()
def lf3(x):
    if x.mdwi <= 0.5465461909770966:
        if x.glcm_ASM <= 0.5137409269809723:
            if x.glcm_dissimilarity <= 0.21074900031089783:
                return WATER
            else:  # if glcm_dissimilarity > 0.21074900031089783
                return LAND
        else:  # if glcm_ASM > 0.5137409269809723
            if x.glcm_contrast <= 0.17211521416902542:
                return WATER
            else:  # if glcm_contrast > 0.17211521416902542
                return LAND
    else:  # if mdwi > 0.5465461909770966
        if x.glcm_ASM <= 0.4484218508005142:
            if x.glcm_dissimilarity <= 0.15898826718330383:
                return LAND
            else:  # if glcm_dissimilarity > 0.15898826718330383
                return WATER
        else:  # if glcm_ASM > 0.4484218508005142
            if x.glcm_energy <= 0.6280775368213654:
                return LAND
            else:  # if glcm_energy > 0.6280775368213654
                return WATER
            
@labeling_function()
def lf4(x):
    if x.glcm_correlation <= 0.9874849617481232:
        if x.glcm_dissimilarity <= 0.21704309433698654:
            if x.glcm_correlation <= 0.6957557499408722:
                return LAND
            else:  # if glcm_correlation > 0.6957557499408722
                return WATER
        else:  # if glcm_dissimilarity > 0.21704309433698654
            if x.ndwi <= 0.5249901413917542:
                return WATER
            else:  # if ndwi > 0.5249901413917542
                return WATER
    else:  # if glcm_correlation > 0.9874849617481232
        return LAND
    
@labeling_function()
def lf5(x):
    if x.rvi <= 0.03211066871881485:
        if x.bi <= 0.46511295437812805:
            if x.glcm_energy <= 0.5166444778442383:
                return WATER
            else:  # if glcm_energy > 0.5166444778442383
                return LAND
        else:  # if bi > 0.46511295437812805
            if x.glcm_homogeneity <= 0.7364725768566132:
                return WATER
            else:  # if glcm_homogeneity > 0.7364725768566132
                return LAND
    else:  # if rvi > 0.03211066871881485
        if x.glcm_ASM <= 0.4935871660709381:
            if x.awei <= 0.4789314568042755:
                return WATER
            else:  # if awei > 0.4789314568042755
                return WATER
        else:  # if glcm_ASM > 0.4935871660709381
            if x.mdwi <= 0.47557683289051056:
                return LAND
            else:  # if mdwi > 0.47557683289051056
                return WATER
        
@labeling_function()
def lf6(x):
    if x.glcm_energy <= 0.6296903491020203:
        if x.glcm_ASM <= 0.37771672010421753:
            if x.glcm_homogeneity <= 0.8294773697853088:
                return WATER
            else:  # if glcm_homogeneity > 0.8294773697853088
                return LAND
        else:  # if glcm_ASM > 0.37771672010421753
            if x.glcm_homogeneity <= 0.8045738935470581:
                return WATER
            else:  # if glcm_homogeneity > 0.8045738935470581
                return LAND
    else:  # if glcm_energy > 0.6296903491020203
        if x.glcm_homogeneity <= 0.7444212436676025:
            if x.glcm_correlation <= 0.41551992297172546:
                return LAND
            else:  # if glcm_correlation > 0.41551992297172546
                return WATER
        else:  # if glcm_homogeneity > 0.7444212436676025
            if x.awei <= 0.11296603083610535:
                return LAND
            else:  # if awei > 0.11296603083610535
                return WATER
            
@labeling_function()
def lf7(x):
    if x.glcm_ASM <= 0.469173327088356:
        if x.glcm_contrast <= 0.10372167825698853:
            if x.ndvi <= 0.44990381598472595:
                return WATER
            else:  # if ndvi > 0.44990381598472595
                return LAND
        else:  # if glcm_contrast > 0.10372167825698853
            if x.mdwi <= 0.7923626005649567:
                return LAND
            else:  # if mdwi > 0.7923626005649567
                return WATER
    else:  # if glcm_ASM > 0.469173327088356
        if x.glcm_energy <= 0.6573227941989899:
            if x.glcm_contrast <= 0.16570807248353958:
                return LAND
            else:  # if glcm_contrast > 0.16570807248353958
                return WATER
        else:  # if glcm_energy > 0.6573227941989899
            if x.glcm_energy <= 0.6701116859912872:
                return LAND
            else:  # if glcm_energy > 0.6701116859912872
                return WATER
            
@labeling_function()
def lf8(x):
    if x.glcm_dissimilarity <= 0.1943829283118248:
        if x.glcm_energy <= 0.5333121418952942:
            if x.glcm_dissimilarity <= 0.15741806477308273:
                return LAND
            else:  # if glcm_dissimilarity > 0.15741806477308273
                return WATER
        else:  # if glcm_energy > 0.5333121418952942
            if x.distance_residential <= 0.041999999433755875:
                return LAND
            else:  # if distance_residential > 0.041999999433755875
                return WATER
    else:  # if glcm_dissimilarity > 0.1943829283118248
        if x.glcm_homogeneity <= 0.7296610474586487:
            if x.mdwi <= 0.7974759638309479:
                return WATER
            else:  # if mdwi > 0.7974759638309479
                return LAND
        else:  # if glcm_homogeneity > 0.7296610474586487
            if x.glcm_energy <= 0.6426526308059692:
                return WATER
            else:  # if glcm_energy > 0.6426526308059692
                return LAND
            
            
@labeling_function()
def lf9(x):
    if x.glcm_ASM <= 0.47108520567417145:
        if x.glcm_dissimilarity <= 0.15456226468086243:
            if x.glcm_dissimilarity <= 0.12170954048633575:
                return LAND
            else:  # if glcm_dissimilarity > 0.12170954048633575
                return WATER
        else:  # if glcm_dissimilarity > 0.15456226468086243
            if x.mdwi <= 0.7968487441539764:
                return WATER
            else:  # if mdwi > 0.7968487441539764
                return LAND
    else:  # if glcm_ASM > 0.47108520567417145
        if x.mdwi <= 0.4028179198503494:
            if x.glcm_homogeneity <= 0.7506296038627625:
                return LAND
            else:  # if glcm_homogeneity > 0.7506296038627625
                return WATER
        else:  # if mdwi > 0.4028179198503494
            if x.rvi <= 0.03531491942703724:
                return WATER
            else:  # if rvi > 0.03531491942703724
                return LAND
            
@labeling_function()
def lf10(x):
    if x.glcm_homogeneity <= 0.7793597877025604:
        if x.glcm_ASM <= 0.5145951509475708:
            if x.mdwi <= 0.7712714374065399:
                return WATER
            else:  # if mdwi > 0.7712714374065399
                return LAND
        else:  # if glcm_ASM > 0.5145951509475708
            if x.mdwi <= 0.3975578844547272:
                return WATER
            else:  # if mdwi > 0.3975578844547272
                return LAND
    else:  # if glcm_homogeneity > 0.7793597877025604
        if x.distance_residential <= 0.041999999433755875:
            if x.ndvi <= 0.4882960170507431:
                return WATER
            else:  # if ndvi > 0.4882960170507431
                return LAND
        else:  # if distance_residential > 0.041999999433755875
            return WATER

@labeling_function()
def lf11(x):
    if x.glcm_contrast <= 0.13594909757375717:
        if x.glcm_energy <= 0.5455110967159271:
            if x.rvi <= 0.016394123435020447:
                return WATER
            else:  # if rvi > 0.016394123435020447
                return LAND
        else:  # if glcm_energy > 0.5455110967159271
            if x.glcm_dissimilarity <= 0.20389101654291153:
                return LAND
            else:  # if glcm_dissimilarity > 0.20389101654291153
                return WATER
    else:  # if glcm_contrast > 0.13594909757375717
        if x.mdwi <= 0.7628937065601349:
            if x.glcm_energy <= 0.6256372630596161:
                return WATER
            else:  # if glcm_energy > 0.6256372630596161
                return LAND
        else:  # if mdwi > 0.7628937065601349
            if x.glcm_ASM <= 0.3288518637418747:
                return WATER
            else:  # if glcm_ASM > 0.3288518637418747
                return LAND
            
@labeling_function()
def lf12(x):
    if x.glcm_ASM <= 0.513109564781189:
        if x.glcm_ASM <= 0.39651769399642944:
            if x.glcm_homogeneity <= 0.836444228887558:
                return WATER
            else:  # if glcm_homogeneity > 0.836444228887558
                return LAND
        else:  # if glcm_ASM > 0.39651769399642944
            if x.glcm_homogeneity <= 0.7893432974815369:
                return WATER
            else:  # if glcm_homogeneity > 0.7893432974815369
                return LAND
    else:  # if glcm_ASM > 0.513109564781189
        if x.glcm_contrast <= 0.15599975734949112:
            return LAND
        else:  # if glcm_contrast > 0.15599975734949112
            if x.distance_residential <= 0.0005000000237487257:
                return LAND
            else:  # if distance_residential > 0.0005000000237487257
                return WATER
            
@labeling_function()
def lf13(x):
    if x.ndvi <= 0.4902210384607315:
        if x.glcm_correlation <= 0.9922609031200409:
            if x.glcm_energy <= 0.5420189499855042:
                return WATER
            else:  # if glcm_energy > 0.5420189499855042
                return LAND
        else:  # if glcm_correlation > 0.9922609031200409
            return LAND
    else:  # if ndvi > 0.4902210384607315
        if x.glcm_ASM <= 0.5222805142402649:
            if x.glcm_ASM <= 0.44570422172546387:
                return LAND
            else:  # if glcm_ASM > 0.44570422172546387
                return WATER
        else:  # if glcm_ASM > 0.5222805142402649
            if x.glcm_dissimilarity <= 0.24923262745141983:
                return LAND
            else:  # if glcm_dissimilarity > 0.24923262745141983
                return WATER
            
@labeling_function()
def lf14(x):
    if x.glcm_energy <= 0.6233303546905518:
        if x.glcm_contrast <= 0.10132023692131042:
            if x.ndwi <= 0.467246875166893:
                return LAND
            else:  # if ndwi > 0.467246875166893
                return WATER
        else:  # if glcm_contrast > 0.10132023692131042
            if x.mdwi <= 0.7584051787853241:
                return LAND
            else:  # if mdwi > 0.7584051787853241
                return WATER
    else:  # if glcm_energy > 0.6233303546905518
        if x.glcm_homogeneity <= 0.7290100157260895:
            if x.glcm_ASM <= 0.5358028411865234:
                return WATER
            else:  # if glcm_ASM > 0.5358028411865234
                return LAND
        else:  # if glcm_homogeneity > 0.7290100157260895
            if x.glcm_homogeneity <= 0.7604183554649353:
                return LAND
            else:  # if glcm_homogeneity > 0.7604183554649353
                return WATER
    
@labeling_function()
def lf15(x):
    if x.glcm_homogeneity <= 0.7849670350551605:
        if x.glcm_homogeneity <= 0.7278034389019012:
            if x.rvi <= 0.006901902379468083:
                return WATER
            else:  # if rvi > 0.006901902379468083
                return LAND
        else:  # if glcm_homogeneity > 0.7278034389019012
            if x.ndmi <= 0.690449982881546:
                return LAND
            else:  # if ndmi > 0.690449982881546
                return WATER
    else:  # if glcm_homogeneity > 0.7849670350551605
        if x.glcm_energy <= 0.582738071680069:
            if x.bi <= 0.2750356048345566:
                return WATER
            else:  # if bi > 0.2750356048345566
                return LAND
        else:  # if glcm_energy > 0.582738071680069
            if x.rvi <= 0.12407626956701279:
                return WATER
            else:  # if rvi > 0.12407626956701279
                return LAND
            
@labeling_function()
def lf16(x):
    if x.glcm_ASM <= 0.49376168847084045:
        if x.mdwi <= 0.778325080871582:
            if x.glcm_ASM <= 0.3756299912929535:
                return LAND
            else:  # if glcm_ASM > 0.3756299912929535
                return LAND
        else:  # if mdwi > 0.778325080871582
            if x.glcm_homogeneity <= 0.749641090631485:
                return LAND
            else:  # if glcm_homogeneity > 0.749641090631485
                return WATER
    else:  # if glcm_ASM > 0.49376168847084045
        if x.bi <= 0.5887406766414642:
            if x.glcm_dissimilarity <= 0.252189502120018:
                return WATER
            else:  # if glcm_dissimilarity > 0.252189502120018
                return LAND
        else:  # if bi > 0.5887406766414642
            if x.glcm_homogeneity <= 0.777024656534195:
                return LAND
            else:  # if glcm_homogeneity > 0.777024656534195
                return WATER
            
@labeling_function()
def lf17(x):
    if x.glcm_ASM <= 0.48306187987327576:
        if x.awei <= 0.6048268675804138:
            if x.glcm_energy <= 0.5761395692825317:
                return LAND
            else:  # if glcm_energy > 0.5761395692825317
                return WATER
        else:  # if awei > 0.6048268675804138
            if x.glcm_ASM <= 0.31013184785842896:
                return LAND
            else:  # if glcm_ASM > 0.31013184785842896
                return WATER
    else:  # if glcm_ASM > 0.48306187987327576
        if x.glcm_energy <= 0.6591232419013977:
            if x.glcm_dissimilarity <= 0.22747769951820374:
                return WATER
            else:  # if glcm_dissimilarity > 0.22747769951820374
                return LAND
        else:  # if glcm_energy > 0.6591232419013977
            if x.glcm_ASM <= 0.566283792257309:
                return WATER
            else:  # if glcm_ASM > 0.566283792257309
                return LAND
            
@labeling_function()
def lf18(x):
    if x.glcm_homogeneity <= 0.8016300201416016:
        if x.glcm_energy <= 0.6281996965408325:
            if x.ndvi <= 0.287639319896698:
                return WATER
            else:  # if ndvi > 0.287639319896698
                return LAND
        else:  # if glcm_energy > 0.6281996965408325
            if x.glcm_contrast <= 0.1654924601316452:
                return WATER
            else:  # if glcm_contrast > 0.1654924601316452
                return LAND
    else:  # if glcm_homogeneity > 0.8016300201416016
        if x.glcm_ASM <= 0.47437405586242676:
            if x.awei <= 0.6300555765628815:
                return LAND
            else:  # if awei > 0.6300555765628815
                return WATER
        else:  # if glcm_ASM > 0.47437405586242676
            return WATER
        
@labeling_function()
def lf19(x):
    if x.rvi <= 0.03185798600316048:
        if x.glcm_energy <= 0.6333625316619873:
            if x.glcm_homogeneity <= 0.8272514343261719:
                return LAND
            else:  # if glcm_homogeneity > 0.8272514343261719
                return WATER
        else:  # if glcm_energy > 0.6333625316619873
            if x.glcm_correlation <= 0.4561297297477722:
                return WATER
            else:  # if glcm_correlation > 0.4561297297477722
                return LAND
    else:  # if rvi > 0.03185798600316048
        if x.bi <= 0.40444037318229675:
            if x.glcm_homogeneity <= 0.8123701214790344:
                return WATER
            else:  # if glcm_homogeneity > 0.8123701214790344
                return LAND
        else:  # if bi > 0.40444037318229675
            if x.glcm_energy <= 0.6285882890224457:
                return LAND
            else:  # if glcm_energy > 0.6285882890224457
                return WATER
            
@labeling_function()
def lf20(x):
    if x.glcm_dissimilarity <= 0.200481079518795:
        if x.glcm_ASM <= 0.42396973073482513:
            if x.awei <= 0.5094348788261414:
                return LAND
            else:  # if awei > 0.5094348788261414
                return WATER
        else:  # if glcm_ASM > 0.42396973073482513
            if x.ndwi <= 0.38722412288188934:
                return LAND
            else:  # if ndwi > 0.38722412288188934
                return WATER
    else:  # if glcm_dissimilarity > 0.200481079518795
        if x.glcm_energy <= 0.6223872303962708:
            if x.awei <= 0.8387463986873627:
                return LAND
            else:  # if awei > 0.8387463986873627
                return WATER
        else:  # if glcm_energy > 0.6223872303962708
            if x.glcm_contrast <= 0.17640645802021027:
                return WATER
            else:  # if glcm_contrast > 0.1764064580202102
                return LAND
            
lfs = [
    lf1,
    lf2,
    lf3,
    lf4,
    lf5,
    lf6,
    lf7,
    lf8,
    lf9,
    lf10,
    lf11,
    lf12,
    lf13,
    lf14,
    lf15,
    lf16,
    lf17,
    lf18,
    lf19,
    lf20
]

applier = PandasLFApplier(lfs)
L_train = applier.apply(y_train)
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, seed=123, lr=0.01, log_freq=10, n_epochs=300)

predicted_labels = label_model.predict(L_train)
print(label_model.score(L_train, y_label, metrics=["f1_micro"])['f1_micro'])